In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from auroraPSI.itx_adapter import ItxAdapter
from auroraPSI.pandas_plotter_adapter import PlotterAdapter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data loading

We load the original Krakow data in format itx, we resample at different resolutions

In [ ]:
itx_adapter = ItxAdapter.read_file("../data/observations/KRK_input_10min.itx")

krakow_dict = itx_adapter.to_pandas()
krakow_df = pd.concat([krakow_dict['data'], krakow_dict['errors']])


In [ ]:
krakow_df.drop(['12.0','37.0','12.0_err','37.0_err'],axis=1,inplace=True)

krakow_df_10min = krakow_df.resample("10min").mean()
krakow_df_30min = krakow_df.resample("30min").mean()
krakow_df_1h = krakow_df.resample("1H").mean()
krakow_df_1d = krakow_df.resample("1D").mean()
krakow_df_1d

In [ ]:
# plotter = PlotterAdapter(krakow_df_10min)
# plotter.scatter_plot()

In [ ]:
# plotter.date_plot()

In [ ]:
# plotter.histogram_plot()

In [ ]:
# plotter.all_plot()

In [ ]:
# plotter.all_error_plot()

In [ ]:
# import scipy.signal as signal
# import numpy as np
# times = krakow_df_10min.index.values.astype(float)
# values = krakow_df_10min['41.0']
# times = times[~np.isnan(values)]
# values = values[~np.isnan(values)]
# w= np.linspace(0.001, 60, krakow_df_10min.shape[0])
# pgram = signal.lombscargle(times,values, w)

In [ ]:
# import matplotlib.pyplot as plt
# fig, (ax_t, ax_w) = plt.subplots(2, 1, constrained_layout=True)
# ax_t.plot(times, values, 'b+')
# ax_t.set_xlabel('Time [s]')
# ax_w.plot(w, pgram)
# ax_w.set_xlabel('Angular frequency [rad/s]')
# ax_w.set_ylabel('Normalized amplitude')
# plt.show()

In [ ]:
values = krakow_df_10min['41.0']
X = krakow_df_10min.values[~np.isnan(values)]

In [ ]:
nt, nx = X.shape # time, compounds
k = 10 #sources 10 to 20

nt, nx

# Define some functions

In [ ]:
from sklearn.decomposition._nmf import _initialize_nmf as initialize_nmf 
from scipy.sparse import lil_matrix, block_diag


dicotomy_tol = 1e-10
shift = 1e-14
sigmaL = 2

def dichotomy_simplex(num, denum, tol=dicotomy_tol, maxit=40):
    """
    Function to solve the num/(x+denum) -1 = 0 equation. Here, x is the Lagragian multiplier which is used to apply the simplex constraint.
    The first part consists in finding a and b such that num/(a+denum) -1 > 0 and num/(b+denum) -1  < 0. 
    The second part applies the dichotomy algorithm to solve the equation.
    """
    # The function has exactly one root at the right of the first singularity (the singularity at min(denum))
    
    # num = num.astype("float64")
    # denum = denum.astype("float64")

    # do some test

    assert((num>=0).all())
    assert((denum>=0).all())
    assert((np.sum(num, axis=0)>0).all())
    
    # Ideally we want to do this, but we have to exclude the case where num==0.
    # a = np.max(num/2 - denum, axis=0)
    if denum.shape[1]>1:
        a = []
        for n,d in zip(num.T, denum.T):
            m = n>0
            a.append(np.max(n[m]/2 - d[m]))
        a = np.array(a)
    else:
        d = denum[:,0]
        def max_masked(n):
            m = n>0
            return np.max(n[m]/2-d[m])
        a = np.apply_along_axis(max_masked, 0, num)
        
        
    # r = np.sum(num/denum, axis=0)
    # b = np.zeros(r.shape)
    # b[r>=1] = (len(num) * np.max(num, axis=0)/0.5 - np.min(denum, axis=0))[r>=1]
    
    b = len(num) * np.max(num, axis=0)/0.5 - np.min(denum, axis=0)

    def func(x):
        return np.sum(num / (x + denum), axis=0) - 1
    func_a = func(a)
    func_b = func(b)
    
    assert(np.sum(func_b>=0)==0)
    assert(np.sum(func_a<=0)==0)
    assert(np.sum(np.isnan(func_a))==0)
    assert(np.sum(np.isnan(func_b))==0)


    return dicotomy(a, b, func, maxit, tol)

def dicotomy(a, b, func, maxit, tol):
    """
    Dicotomy algorithm searching for func(x)=0.

    Inputs:
    * a: bound such that func(a) > 0
    * b: bound such that func(b) < 0
    * maxit: maximum number of iteration
    * tol: tolerance - the algorithm stops if |func(sol)| < tol
    
    This algorithm work for number or numpy array of any size.
    """
    # Dichotomy algorithm to solve the equation
    it = 0
    new = (a + b)/2
    func_new = func(new)
    while np.max(np.abs(func_new)) > tol:
        
        it=it+1
        func_a = func(a)
        # func_b = func(b)

        # if f(a)*f(new) <0 then f(new) < 0 --> store in b
        minus_bool = func_a * func_new <= 0
        
        # if f(a)*f(new) > 0 then f(new) > 0 --> store in a
        # plus_bool = func_a * func_new > 0
        plus_bool = np.logical_not(minus_bool)

        b[minus_bool] = new[minus_bool]
        a[plus_bool] = new[plus_bool]
        new = (a + b) / 2
        func_new = func(new)
        if it>=maxit:
            print("Dicotomy stopped for maximum number of iterations")
            break
    return new

def Frobenius_loss(X, W, H, average=False):
    """
    Compute the generalized KL divergence.

    \sum_{ji} | X_{ij} - (D H)_{ij} |^2
    """
    
    DH = W @ H

    if average:
        return np.mean((DH - X)**2)
    else:
        return np.sum((DH - X)**2)

def trace_xtLx(L, x, average=False):
    if average:
        return np.mean(x * (L @ x))
    else:
        return np.sum(x * (L @ x))
    
    

def create_laplacian_matrix(nx):
    """
    Helper method to create the laplacian matrix for the laplacian regularization
    :param n: width of the original image
    :return:the n x n laplacian matrix
    """
    assert(nx>1)
    #Blocks corresponding to the corner of the image (linking row elements)
    mat=lil_matrix((nx,nx),dtype=np.float32)
    mat.setdiag(2)
    mat.setdiag(-1,k=1)
    mat.setdiag(-1,k=-1)
    mat[0,0] = 1
    mat[nx-1, nx-1] = 1
    return mat

def multiplicative_step_G(X, F, G, force_simplex=True,  safe=True, dicotomy_tol=dicotomy_tol, shift=shift):
    """
    Multiplicative step in G.
    """

    if safe:
        # Allow for very small negative values!
        assert(np.sum(G<-shift/2)==0)
        assert(np.sum(F<-shift/2)==0)
        G = np.maximum(G, shift)
        F = np.maximum(F, shift)
    
    FF = F.T @ F
    FX = F.T @ X
    num = G * FX
    denum = FF @ G


    if force_simplex:
        nu = dichotomy_simplex(num, denum,dicotomy_tol)
    else:
        nu = 0
        
    if safe:
        assert np.sum(denum<0)==0
        assert np.sum(num<0)==0

    new_G = num/(denum+nu)
    return new_G



def multiplicative_step_F(X, F, G, lambda_L=0, L=None, sigmaL=sigmaL, safe=True, shift=shift):
    """
    Multiplicative step in F.
    """
    if not(lambda_L==0):
        if L is None:
            raise ValueError("Please provide the laplacian")
    if safe:
        # Allow for very small negative values!
        assert(np.sum(G<-shift/2)==0)
        assert(np.sum(F<-shift/2)==0)
        G = np.maximum(G, shift)
        F = np.maximum(F, shift)

    GG = G @ G.T
    GX = X @ G.T
    num = GX
    denum = F @ GG  


    if not(lambda_L==0):
        num = num + lambda_L * sigmaL
        denum = denum + lambda_L * sigmaL + lambda_L * L @ F 
        
    num = F * num
    
    if safe:
        assert np.sum(denum<0)==0
        assert np.sum(num<0)==0

    new_F = num/denum

    return new_F

def initialize_algorithms(X, F, G, n_components, init, random_state, force_simplex):
    # Handle initialization
    if F is None:
        if G is None:
            F, G = initialize_nmf(X, n_components=n_components, init=init, random_state=random_state)
            if force_simplex:
                scale = np.sum(G, axis=0, keepdims=True)
                G = np.nan_to_num(G/scale, nan = 1.0/G.shape[0] )
        else:
            F = np.abs(np.linalg.lstsq(G, X.T, rcond=None)[0]).T

    elif G is None:
        G = np.abs(np.linalg.lstsq(F, X, rcond=None)[0])
        if force_simplex:
            scale = np.sum(G, axis=0, keepdims=True)
            G = G/scale

    return F, G


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

import time

class NMFEstimator(TransformerMixin, BaseEstimator):
    
    loss_names_ = ["Frobenius loss", "Laplacian loss"]

    
    def __init__(self, n_components=2, init='warn', tol=1e-4, max_iter=200,
                 random_state=None, verbose=1, debug=False, lambda_L=0,
                shift=shift, eval_print=10, force_simplex=False, dicotomy_tol=dicotomy_tol):
        self.n_components = n_components
        self.init = init
        self.tol = tol
        self.max_iter = max_iter
        self.random_state = random_state
        self.verbose = verbose
        self.shift = shift
        self.debug = debug
        self.eval_print = eval_print
        self.lambda_L = lambda_L
        self.force_simplex = force_simplex
        self.dicotomy_tol = dicotomy_tol



    def _more_tags(self):
        return {'requires_positive_X': True}

    def _iteration(self,  F, G):
        F = multiplicative_step_F(self.X_,  F, G, safe=self.debug, lambda_L=self.lambda_L, L=self.L_, shift=self.shift)
        G = multiplicative_step_G(self.X_, F, G, force_simplex=self.force_simplex, safe=self.debug, dicotomy_tol=self.dicotomy_tol, shift=self.shift)
        return F, G
    

    def loss(self, F, G, average=True, X = None):
        if X is None : 
            X = self.X_

        assert(X.shape == (F.shape[0],G.shape[1]))

        self.FG_numel_ = F.shape[0] * G.shape[1]
        
        loss_fro = Frobenius_loss(X, F, G, average=average)
        loss = loss_fro
        self.detailed_loss_ = [loss_fro, 0]

        if self.lambda_L>0:
            loss_tr =  self.lambda_L * trace_xtLx(self.L_, F, average=average)
            loss += loss_tr
            self.detailed_loss_[1] = loss_tr
        
        # if average:
        #     loss = loss / self.FG_numel_
        return loss

    def fit_transform(self, X, y=None, F=None, G=None):
        """Learn a NMF model for the data X and returns the transformed data.
        This is more efficient than calling fit followed by transform.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data matrix to be decomposed
        F : array-like of shape (n_samples, n_components)
            If specified, it is used as initial guess for the solution.
        G : array-like of shape (n_components, n_features)
            If specified, it is used as initial guess for the solution.
        Returns
        -------
        F,G : ndarrays
        """

        self.X_ = self._validate_data(X, dtype=[np.float64, np.float32])

  


        self.X_ = self.remove_zeros_lines(self.X_, self.shift)

        
        self.F_, self.G_ = initialize_algorithms(X = self.X_, F= F, G = G, n_components = self.n_components, init = self.init, random_state = self.random_state, force_simplex = self.force_simplex)

        if self.lambda_L > 0:
            self.L_ = create_laplacian_matrix(self.F_.shape[0])
        else:
            self.L_ = None


        algo_start = time.time()
        # If mu_sparse != 0, this is the regularized step of the algorithm
        # Otherwise this is directly the data fitting step
        eval_before = np.inf
        eval_init = self.loss(self.F_, self.G_)
        self.n_iter_ = 0

        # if self.debug:
        self.losses_ = []
        self.rel_ = []
        self.detailed_losses_ = []
        
        try:
            while True:
                # Take one step in A, P
                old_F, old_G = self.F_.copy(), self.G_.copy()
                
                self.F_, self.G_ = self._iteration(self.F_, self.G_ )
                eval_after = self.loss(self.F_, self.G_)
                self.n_iter_ +=1
                
                rel_F = np.max((self.F_ - old_F)/(self.F_ + self.tol*np.mean(self.F_) ))
                rel_G = np.max((self.G_ - old_G)/(self.G_ + self.tol*np.mean(self.G_) ))

                # store some information for assessing the convergence
                # for debugging purposes
                detailed_loss_ = self.detailed_loss_

                self.losses_.append(eval_after)
                self.detailed_losses_.append(detailed_loss_)
                self.rel_.append([rel_F,rel_G])
                              
                # check convergence criterions
                if self.n_iter_ >= self.max_iter:
                    print("exits because max_iteration was reached")
                    break

                # If there is no regularization the algorithm stops with this criterion
                # Otherwise it goes to the data fitting step
                elif max(rel_F,rel_G) < self.tol:
                    print(
                        "exits because of relative change rel_F {} or rel_G {} < tol ".format(
                            rel_F,rel_G
                        )
                    )
                    break
                elif abs((eval_before - eval_after)/eval_init) < self.tol:
                    print(
                        "exits because of relative change < tol: {}".format(
                            (eval_before - eval_after)/min(eval_before, eval_after)
                        )
                    )
                    break

                elif np.isnan(eval_after):
                    print("exit because of the presence of NaN")
                    break

                elif (eval_before - eval_after) < 0:
                    if hasattr(self, "accelerate"):
                        if not self.accelerate:
                            print("exit because of negative decrease {}: {}, {}".format((eval_before - eval_after), eval_before, eval_after))
                            break
                    else:
                        print("exit because of negative decrease {}: {}, {}".format((eval_before - eval_after), eval_before, eval_after))
                        break
                
                if self.verbose > 0 and np.mod(self.n_iter_, self.eval_print) == 0:
                    print(
                        f"It {self.n_iter_} / {self.max_iter}: loss {eval_after:0.3f},  {self.n_iter_/(time.time()-algo_start):0.3f} it/s",
                    )
                    pass
                eval_before = eval_after
        except KeyboardInterrupt:
            pass
        
        # if not(self.force_simplex):
        #     self.F_, self.G_ = rescaled_DH(self.F_, self.G_ )
        
        algo_time = time.time() - algo_start
        print(
            f"Stopped after {self.n_iter_} iterations in {algo_time//60} minutes "
            f"and {np.round(algo_time) % 60} seconds."
        )
        self.reconstruction_err_ = self.loss(self.F_, self.G_)
        
        self.n_components_ = self.G_.shape[0]
        self.components_ = self.G_
        return self.F_

    def fit(self, X, y=None, **params):
        """Learn a NMF model for the data X.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data matrix to be decomposed
        y : Ignored
        Returns
        -------
        self
        """
        self.fit_transform(X, **params)
        return self

    def inverse_transform(self, F):
        """Transform data back to its original space.
        Parameters
        ----------
        W : {ndarray, sparse matrix} of shape (n_samples, n_components)
            Transformed data matrix.
        Returns
        -------
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            Data matrix of original shape.
        .. versionadded:: 0.18
        """
        check_is_fitted(self)
        return  F @ self.G_
    
    def get_losses(self):
        
        names = ["full_loss"] + self.loss_names_ + ["rel_W","rel_H"]
        dt_list = []
        for elt in names : 
            dt_list.append((elt,"float64"))
        dt = np.dtype(dt_list)

        tup_list = []
        for i in range(len(self.losses_)) : 
            tup_list.append((self.losses_[i],) + tuple(self.detailed_losses_[i]) + tuple(self.rel_[i]))
        
        array = np.array(tup_list,dtype=dt)

        return array

    def remove_zeros_lines (self, X, epsilon) : 
        if np.all(X >= 0) : 
            new_X = X.copy()
            sum_cols = X.sum(axis = 0)
            sum_rows = X.sum(axis = 1)
            new_X[:,np.where(sum_cols == 0)] = epsilon
            new_X[np.where(sum_rows == 0),:] = epsilon
            return new_X
        else : 
            raise ValueError("There are negative values in X")

In [ ]:

plt.imshow(X[:200])
plt.colorbar()
np.min(X)

In [ ]:
plt.imshow(X[:200]<0)

In [ ]:

est = NMFEstimator(n_components=k
             , init='random', tol=1e-4, max_iter=200,
                 random_state=None, verbose=1, debug=False, lambda_L=3,
                shift=shift, eval_print=10, force_simplex=False)

In [ ]:
Xnew = X.copy()
Xnew[X<0] = 0
est.fit(Xnew)

In [ ]:
F = est.F_
G = est.G_
loss = est.get_losses()

In [ ]:
# np.sum(G, axis=0)

In [ ]:
import re
mark_space = 20
fontsize = 15
marker_list = ["-o","-s","->","-<","-^","-v","-d"]


fig, axes = plt.subplots(1, 2, figsize=(15, 5))

names = list(loss.dtype.names)
for j,name in enumerate(names) :
    if re.match(r".*(loss)",name) : 
        axes[0].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[0].set_yscale("log")
        axes[0].legend()
        axes[0].set_xlabel("number of iterations")
    elif re.match(r"^(rel)",name) : 
        axes[1].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[1].legend()
        axes[1].set_xlabel("number of iterations")
    # elif re.match(r"^(ang)",name) :
    #     axes[2].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
    #     axes[2].legend()
    #     axes[2].set_xlabel("number of iterations")
    # elif re.match(r"^(mse)",name) :
    #     axes[3].plot(loss[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
    #     axes[3].legend()
    #     axes[3].set_xlabel("number of iterations")

cols = ["Losses", "Evolution of A and P","Angles","MSE"]

for ax, col in zip(axes, cols):
    ax.set_title(col, fontsize=fontsize)

fig.tight_layout()

In [ ]:
plt.plot(F)

In [ ]:
plt.imshow(G)
plt.colorbar()